Vediamo diversi casi d'uso per le LSTM, di complessità crescente

# Time series univariate

Sono il tipo di time series più semplice, nel tempo è raccolto il valore di una sola variabile.

Supponiamo di avere la seguente time series:

<table>
  <tr>
    <th>Valore 1</th>
  </tr>
  <tr>
    <td>10</td>
  </tr>
  <tr>
    <td>20</td>
  </tr>
  <tr>
    <td>30</td>
  </tr>
  <tr>
    <td>40</td>
  </tr>
  <tr>
    <td>50</td>
  </tr>
  <tr>
    <td>60</td>
  </tr>
  <tr>
    <td>70</td>
  </tr>
  <tr>
    <td>80</td>
  </tr>
  <tr>
    <td>90</td>
  </tr>
</table>

Ponendo un time steps uguale a 3, abbiamo il nostro dataset in questo modo:

il time step è la sottosequenza temporale da considerare quando vogliamo predire un'azione, un qualcosa che si svolge in una sequenza di tempo

In [1]:
X = [[10, 20, 30], 
     [20, 30, 40],
     [30, 40, 50],
     [40, 50, 60],
     [50, 60, 70],
     [60, 70, 80]]

# abbiamo preso in input, dunque, tutte le sottosequenze di tre, dove ognuna è formata da valori uno successivo all'altro

y = [40,
     50,
     60,
     70,
     80,
     90]

# y: output, per ogni sottosequenza in input (es. 40 predizione in output della sottosequenza [10, 20, 30])

Costruiamo ora una semplice LSTM

In [2]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

time_steps = 3
num_features = 1 # perchè stiamo analizzando le time series univariate

# trasformiamo dataset (input e output) in array
X = np.array(X)
y = np.array(y)

# ridimensionamento: input deve avere dimensione: num_samples (o batch_size), time_steps, num_features
# (in questo caso rispettivamente: 6 (numero sottosequenze), 3 (time step), 1 (time series univariate))
X = X.reshape(X.shape[0], X.shape[1], num_features)

print(X.shape, y.shape)

# il modello Sequential si usa quando si vuole avere una sequenza di layer, ognuno con un singolo input/output
model = Sequential()

# aggiungiamo il primo layer, di 50 neuroni, LSTM, in cui definiamo la dimensione dell'input (3, 1)
model.add(LSTM(50, activation = "relu", input_shape = (time_steps, num_features)))

# aggiungiamo il secondo layer, Dense, che servirà per la predizione (1 output)
# N.B. in un Dense layer un neurone riceve input da tutti i neuroni del layer precedente
model.add(Dense(1))

# addestriamo usando Adam e utilizziamo come Loss function l'Errore Quadratico
model.compile(optimizer = "adam", loss = "mse")

# addestriamo per 300 epoche: significa che il dataset sarà "passato" sotto l'algoritmo di apprendimento 300 volte
model.fit(X, y, epochs = 300, verbose = 0)

(6, 3, 1) (6,)


E testiamo il modello

In [3]:
X_test = np.array([70, 80, 90]) # ci aspettiamo 100
# X_test = np.array([100, 110, 120]) # ci aspettiamo 130
X_test = X_test.reshape((1, time_steps, num_features))

y_test = model.predict(X_test)
print(y_test)

1/1 [==============================] - 1s 530ms/step
[[102.52359]]


# Time series multivariate

In questo tipo di time series, nel tempo sono raccolte più variabili.

Vedremo due tipi di problemi con questo tipo di time series:


1.   Raccolte due o più serie temporali, vogliamo predire il valore di una terza serie di valori
2.   Raccolte due o più serie temporali, vogliamo predire lo step successivo


Per quanto riguarda il primo problema, supponiamo di aver raccolto i seguenti valori:



<table>
  <tr>
    <th>Valore 1</th>
    <th>Valore 2</th>
    <th>Output</th>
  </tr>
  <tr>
    <td>10</td>
    <td>15</td>
    <td>25</td>
  </tr>
  <tr>
    <td>20</td>
    <td>25</td>
    <td>45</td>
  </tr>
  <tr>
    <td>30</td>
    <td>35</td>
    <td>65</td>
  </tr>
  <tr>
    <td>40</td>
    <td>45</td>
    <td>85</td>
  </tr>
  <tr>
    <td>50</td>
    <td>55</td>
    <td>105</td>
  </tr>
  <tr>
    <td>60</td>
    <td>65</td>
    <td>125</td>
  </tr>
  <tr>
    <td>70</td>
    <td>75</td>
    <td>145</td>
  </tr>
  <tr>
    <td>80</td>
    <td>85</td>
    <td>165</td>
  </tr>
  <tr>
    <td>90</td>
    <td>95</td>
    <td>185</td>
  </tr>
</table>

Andiamo a ripetere il processo di preparazione dei dati visto in precedenza;
in questo caso, però, avremo una tripla di valori, e non un singolo valore, come input.
Impostando, ad esempio, un time steps uguale a 3, otteniamo il seguente dataset








In [4]:
X = [[[10, 15],
      [20, 25],
      [30, 35]],
     [[20, 25],
      [30, 35],
      [40, 45]],
     [[30, 35],
      [40, 45],
      [50, 55]],
     [[40, 45],
      [50, 55],
      [60, 65]],
     [[50, 55],
      [60, 65],
      [70, 75]],
     [[60, 65],
      [70, 75],
      [80, 85]],
     [[70, 75],
      [80, 85],
      [90, 95]]]

y = [65,
     85,
     105,
     125,
     145,
     165,
     185]

X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)

(7, 3, 2) (7,)


Andiamo a costruire il nostro modello

In [5]:
time_steps = 3
num_features = 2

model = Sequential()
model.add(LSTM(50, activation="relu", input_shape=(time_steps, num_features)))
model.add(Dense(1))
model.compile(optimizer="adam", loss="mse")

model.fit(X, y, epochs=300, verbose=0)

Ed effettuiamo la predizione su un nuovo input

In [6]:
X_test = np.array([[80, 85], [90, 95], [100, 105]])
X_test = X_test.reshape((1, time_steps, num_features))

y_test = model.predict(X_test)
print(y_test)

1/1 [==============================] - 0s 443ms/step
[[206.17143]]


### Adesso affrontiamo il secondo tipo di problema, ovvero la predizione del prossimo step in una time series multivariata.

Il problema è dello stesso tipo del primo caso visto, quello per le time series univariate.
Supponiamo di aver raccolto i seguenti dati osservando tre variabili differenti:

<table>
  <tr>
    <th>Valore 1</th>
    <th>Valore 2</th>
    <th>Valore 3</th>
  </tr>
  <tr>
    <td>10</td>
    <td>15</td>
    <td>25</td>
  </tr>
  <tr>
    <td>20</td>
    <td>25</td>
    <td>45</td>
  </tr>
  <tr>
    <td>30</td>
    <td>35</td>
    <td>65</td>
  </tr>
  <tr>
    <td>40</td>
    <td>45</td>
    <td>85</td>
  </tr>
  <tr>
    <td>50</td>
    <td>55</td>
    <td>105</td>
  </tr>
  <tr>
    <td>60</td>
    <td>65</td>
    <td>125</td>
  </tr>
  <tr>
    <td>70</td>
    <td>75</td>
    <td>145</td>
  </tr>
  <tr>
    <td>80</td>
    <td>85</td>
    <td>165</td>
  </tr>
  <tr>
    <td>90</td>
    <td>95</td>
    <td>185</td>
  </tr>
</table>

Considerando sempre un time steps di 3, possiamo procedere con la preparazione dei dati e ottenere il seguente dataset

In [7]:
X = [[[10, 15, 25],
      [20, 25, 45],
      [30, 35, 65]],
     [[20, 25, 45],
      [30, 35, 65],
      [40, 45, 85]],
     [[30, 35,  65],
      [40, 45,  85],
      [50, 55, 105]],
     [[40, 45,  85],
      [50, 55, 105],
      [60, 65, 125]],
     [[50, 55, 105],
      [60, 65, 125],
      [70, 75, 145]],
     [[60, 65, 125],
      [70, 75, 145],
      [80, 85, 165]]]

y = [[40, 45, 85],
     [50, 55, 105],
     [60, 65, 125],
     [70, 75, 145],
     [80, 85, 165],
     [90, 95, 185]]

X = np.array(X)
y = np.array(y)

print(X.shape, y.shape)

(6, 3, 3) (6, 3)


Allo stesso modo visto in precedenza, costruiamo il modello e andiamo a predire su un nuovo esempio

In [8]:
time_steps = 3
num_features = 3

model = Sequential()
model.add(LSTM(50, activation="relu", input_shape=(time_steps, num_features)))
model.add(Dense(num_features))
model.compile(optimizer="adam", loss="mse")

model.fit(X, y, epochs=300, verbose=0)

In [9]:
X_test = np.array([[70, 75, 145], [80, 85, 165], [90, 95, 185]])
X_test = X_test.reshape((1, time_steps, num_features))

y_test = model.predict(X_test)
print(y_test)

1/1 [==============================] - 0s 430ms/step
[[101.04109 105.71552 207.24236]]
